# Download Europe OSM PBF to Snowflake Stage
This notebook downloads the entire Europe OSM PBF file from Geofabrik and uploads it to the Snowflake internal stage. Warning: This is a very large file (~30GB) and will take significant time to download and upload.

In [ ]:
# Install required packages
!pip install requests snowflake-snowpark-python 

In [ ]:
import requests
import os
import sys
from snowflake.snowpark import Session

country = sys.argv[0]

url = f"https://download.geofabrik.de/europe/{country}-latest.osm.pbf"
local_file = f"/tmp/{country}-latest.osm.pbf"

print(f"Downloading {country.upper()} OSM PBF file...")
print(f"URL: {url}")
print("Downloading file...")

# Download the file with progress
response = requests.get(url, stream=True)

with open(local_file, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192000):
        if chunk:
            f.write(chunk)

print(f"\nDownload complete! File saved to: {local_file}")
print(f"File size: {os.path.getsize(local_file):,} bytes")

In [ ]:
# Get current Snowflake session
session = Session.builder.getOrCreate()

print("Uploading to Snowflake stage...")
stage_location = "@openrouteservice_native_app.core.ors_spcs_stage"

# Upload file to stage
put_result = session.file.put(
    local_file_name=local_file,
    stage_location=stage_location,
    auto_compress=False,
    overwrite=True
)

print("\nUpload complete!")
print(put_result)

In [ ]:
# Verify the file is in the stage
print("\nVerifying file in stage...")
stage_files = session.sql(f"LIST {stage_location}").collect()

for file in stage_files:
    if 'albania' in file['name'].lower():
        print(f"✓ Found: {file['name']}")
        print(f"  Size: {file['size'] / (1024**3):.2f} GB")

In [ ]:
# Clean up local file
if os.path.exists(local_file):
    os.remove(local_file)
    print(f"\nCleaned up local file: {local_file}")